# Collecting the dataset

In [3]:
import os 
import urllib
import urllib.request 
from zipfile import ZipFile
import numpy as np
import cv2


URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'FashionMNISTImage'

if not os.path.isfile(FILE): #Checks to see if the file already exists 
    urllib.request.urlretrieve(URL, FILE) #Download the image 

if not os.path.isdir(FOLDER):
    with ZipFile(FILE) as zippedImages:
        zippedImages.extractall(FOLDER) #unzips the folder 

def loadData(dataset, path):
    #path refers to the folder
    #path refers to the batches of data 
    labels = os.path.join(path, dataset)

    X = []
    y = []
    for folderClass in os.listdir(labels):
        #folderClass contains the class classification
        for images in os.listdir(os.path.join(path, dataset, folderClass)):
            readImage = cv2.imread(os.path.join(path, dataset, folderClass, images), cv2.IMREAD_UNCHANGED)
            #IMREAD Makes it read a grey scale from -1 to 1 (The image data has no colours)
            X.append(readImage)
            y.append(folderClass)
    return np.array(X), np.array(y, dtype = np.uint8)

def createData(path):
    X,y = loadData('train', path)
    Xtest, ytest = loadData('test', path)

    return X, y, Xtest, ytest

X, y, Xtest, ytest = createData(FOLDER)


keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)

X = X[keys]
y = y[keys]



X = (X.reshape(X.shape[0], -1).astype(np.float32) - 127.5) /127.5
Xtest = (Xtest.reshape(Xtest.shape[0], -1).astype('float32') - 127.5) /127.5

print(Xtest.shape)
print(X.shape)


(10000, 784)
(60000, 784)


## Training the neural network

In [4]:

import Neural_network_code.neuroNetwork as nn
import Neural_network_code.Optimizers as op 

#The reshaping exists to turn the cv2 conversion, which presents itself in rows of rows into a single row and the normalizing is\
#to make values between -1 and 0 for which neuralnetworks work best with 

model = nn.NeuroModel()

model.add(nn.neuroNetworkLayer(X.shape[1], 128))
model.add(nn.activateRELU())
model.add(nn.neuroNetworkLayer(128, 128))
model.add(nn.activateRELU())
model.add(nn.neuroNetworkLayer(128, 10))
model.add(nn.softMaxFunction())
model.set(nn.LossCategoricalCrossEntropy(), op.Optimizer_Adam(rate = 0.001, decay = 1e-3), nn.categoricalAccuracy())
model.finalize()

model.train(X, y, 10, 1,  batchSize = 128)
model.evaluate(Xtest, ytest, batchSize = 128)




s 0
p 0.0703125
l 2.3027586936950684
nl 2.3027587
rl 0.0


s 100
p 0.734375
l 0.6662283539772034
nl 0.66622835
rl 0.0


s 200
p 0.7734375
l 0.655479371547699
nl 0.6554794
rl 0.0


s 300
p 0.796875
l 0.4865652322769165
nl 0.48656523
rl 0.0


s 400
p 0.828125
l 0.4579637050628662
nl 0.4579637
rl 0.0


Validation
e 0
p 0.7511852297008547
l 0.6622782168734787
nl 0.6622782168734787
rl 0.0


s 0
p 0.78125
l 0.6522002220153809
nl 0.6522002
rl 0.0


s 100
p 0.8359375
l 0.4055500328540802
nl 0.40555003
rl 0.0


s 200
p 0.796875
l 0.5585809350013733
nl 0.55858094
rl 0.0


s 300
p 0.84375
l 0.4104033410549164
nl 0.41040334
rl 0.0


s 400
p 0.84375
l 0.3841607868671417
nl 0.3841608
rl 0.0


Validation
e 1
p 0.8419471153846154
l 0.4338723335128564
nl 0.4338723335128564
rl 0.0


s 0
p 0.8359375
l 0.5514023303985596
nl 0.55140233
rl 0.0


s 100
p 0.8515625
l 0.3653751611709595
nl 0.36537516
rl 0.0


s 200
p 0.8125
l 0.5069138407707214
nl 0.50691384
rl 0.0


s 300
p 0.859375
l 0.3791733980178833
nl 0.

## Saving the network

In [5]:
import pickle 

with open('SavedModel\\NeuroNetworkModel.pickle', 'wb') as file:
    pickle.dump(model, file)
